# Imports

In [1]:
import pandas as pd
from groq import Groq
import templates
import json

# Constants

In [2]:
MODEL_NAME = "llama3-8b-8192"
DATA_PATH_REGULAR = "./experimental_data/UMWPDataset_last.csv"
DATA_PATH_PROBE = "./experimental_data/UMWPDataset_probe_last.csv"
TOTAL_EVAL = 100
EVAL_PATH = "./evaluations/UMWP_latter.json"

# Load Data

In [3]:
data_reg = pd.read_csv(DATA_PATH_REGULAR)
data_probe = pd.read_csv(DATA_PATH_PROBE)
print(len(data_reg), len(data_probe))

100 100


# Explore data

In [10]:
c = 63
print("question:              " + str(data_reg["question"][c]))
print("golden example:        " + str(data_reg["answer"][c]))
print("answerable:            " + str(data_reg["answerable"][c]))
# print("false example:         " + str(data_reg["Examples: False"][c]))
# print("uninformative example: " + str(data_reg["Examples: Uninformative"][c]))
# print("informative example:   " + str(data_reg["Examples: Informative"][c]))
print("*" * 50)
print("one agent result:      " + str(data_reg["single_agent"][c]))
print("*" * 50)
# print("feedback1:             " + str(data_reg["feedback1"][c]))
print("*" * 50)
# print("feedback1 probing:     " + str(data_probe["feedback1"][c]))
print("*" * 50)
print("two agents result:     " + str(data_reg["two_agents"][c]))
print("*" * 50)
print("two agents probing:    " + str(data_probe["two_agents"][c]))
print("*" * 50)
# print("three agents result:   " + str(data_reg["three_agents"][c]))
print("*" * 50)
print("three agents probing:  " + str(data_probe["three_agents"][c]))

question:              The ages of two friends Anna and Ben differ by 3 years. Anna's father Denver is twice as old as Anna and Ben is twice as old as his sister Cathy. Find the ages of Anna and Ben.
golden example:        nan
answerable:            False
**************************************************
one agent result:      

**Solution:**

Let's break down the problem step-by-step:

**1. Define variables:**

*  Let Anna's age be 'A'
*  Let Ben's age be 'B'
*  Let Denver's age be 'D'
*  Let Cathy's age be 'C'

**2. Set up equations based on the given information:**

*  A - B = 3  (Anna and Ben's ages differ by 3 years)
*  D = 2A (Denver is twice as old as Anna)
*  B = 2C (Ben is twice as old as his sister Cathy)

**3. Solve for the ages:**

*  We can use the first equation to express 'A' in terms of 'B':  A = B + 3
*  Substitute this value of 'A' into the second equation: D = 2(B + 3) 
*  Substitute the value of 'B' from the third equation into the second equation: D = 2(2C + 3)
* 

# Evaluation

Groq API key: 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

In [5]:
client = Groq(api_key="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")
results = []
for i in range(63, TOTAL_EVAL):
    result = {}
    prompt = templates.Evaluation.UMWP_EVALUATION_PROMPT_TEMPLATE.substitute(
        answerable=data_reg["answerable"], answer=data_reg["answer"][i], question=data_reg["question"][i], response=data_reg["single_agent"][i]
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "you are doing evaluation to a language model."
            },
            # Set a user message for the assistant to respond to.
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-70b-8192"
    )
    result["single_agent"] = chat_completion.choices[0].message.content
    
    prompt = templates.Evaluation.UMWP_EVALUATION_PROMPT_TEMPLATE.substitute(
        answerable=data_reg["answerable"], answer=data_reg["answer"][i], question=data_reg["question"][i], response=data_reg["two_agents"][i]
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "you are doing evaluation to a language model."
            },
            # Set a user message for the assistant to respond to.
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-70b-8192"
    )
    result["two_agents"] = chat_completion.choices[0].message.content
    
    prompt = templates.Evaluation.UMWP_EVALUATION_PROMPT_TEMPLATE.substitute(
        answerable=data_reg["answerable"], answer=data_reg["answer"][i], question=data_reg["question"][i], response=data_reg["three_agents"][i]
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "you are doing evaluation to a language model."
            },
            # Set a user message for the assistant to respond to.
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-70b-8192"
    )
    result["three_agents"] = chat_completion.choices[0].message.content
    
    prompt = templates.Evaluation.UMWP_EVALUATION_PROMPT_TEMPLATE.substitute(
        answerable=data_reg["answerable"], answer=data_reg["answer"][i], question=data_reg["question"][i], response=data_probe["two_agents"][i]
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "you are doing evaluation to a language model."
            },
            # Set a user message for the assistant to respond to.
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-70b-8192"
    )
    result["two_agents_probing"] = chat_completion.choices[0].message.content
    
    prompt = templates.Evaluation.UMWP_EVALUATION_PROMPT_TEMPLATE.substitute(
        answerable=data_reg["answerable"], answer=data_reg["answer"][i], question=data_reg["question"][i], response=data_probe["three_agents"][i]
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "you are doing evaluation to a language model."
            },
            # Set a user message for the assistant to respond to.
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-70b-8192"
    )
    result["three_agents_probing"] = chat_completion.choices[0].message.content
    
    results.append(result)
    
    with open(EVAL_PATH, "w") as f:
        f.write(json.dumps(results, indent=4))

APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `llama3-70b-8192` in organization `org_01jc91p2p5e5rt3pvcsh8t48nm` on tokens per minute (TPM): Limit 6000, Requested 24986, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}